<a href="https://colab.research.google.com/github/javiercarpio57/03MIAR---Algoritmos-de-Optimizacion/blob/main/Algoritmos_Hector_Javier_Carpio_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 3

<strong>Nombre: Héctor Javier Carpio García</strong>


https://colab.research.google.com/drive/1otTvQgj5rdFPME_IuPxFrGWkTh7Fw7PX?usp=sharing

https://github.com/javiercarpio57/03MIAR---Algoritmos-de-Optimizacion.git

In [2]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


#Carga de los datos del problema

In [3]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios


#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)

In [4]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [5]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
                
#dir(problem)

15

In [6]:
 
#Funcionas basicas
###############################################################################      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i], solucion[i+1], problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#BÚSQUEDA ALEATORIA

In [7]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion

In [8]:
#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 19, 33, 34, 2, 23, 40, 28, 10, 16, 8, 27, 4, 14, 36, 35, 5, 7, 17, 20, 29, 39, 9, 25, 12, 32, 38, 24, 41, 21, 18, 37, 6, 3, 31, 30, 15, 13, 26, 11, 1, 22]
Distancia     : 3797


#BUSQUEDA LOCAL

In [9]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [10]:
#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3797
Distancia Mejor Solucion Local: 3482


In [15]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(solucion_inicial, problem):
  mejor_solucion = []

  mejor_distancia = distancia_total(solucion_inicial, problem)
  iteracion = 0             #Un contador para saber las iteraciones que hacemos
  while 1:
    iteracion += 1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_inicial)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion
      
    solucion_inicial = vecina

In [16]:
#Generar una solucion inicial de referencia(aleatoria)
solucion_referencia = crear_solucion(Nodos)

sol = busqueda_local(solucion_referencia, problem)

En la iteracion  38 , la mejor solución encontrada es: [0, 17, 31, 35, 36, 37, 15, 16, 14, 19, 13, 5, 26, 6, 7, 1, 4, 3, 2, 27, 28, 8, 29, 30, 32, 20, 33, 34, 38, 22, 39, 9, 25, 11, 12, 18, 10, 41, 23, 40, 24, 21]
Distancia     : 1619


In [17]:
print(sol)

[0, 17, 31, 35, 36, 37, 15, 16, 14, 19, 13, 5, 26, 6, 7, 1, 4, 3, 2, 27, 28, 8, 29, 30, 32, 20, 33, 34, 38, 22, 39, 9, 25, 11, 12, 18, 10, 41, 23, 40, 24, 21]


###Búsqueda local con entorno variables
####Búsqueda con Multi-arranque

In [18]:
def busqueda_multiarranque(problem, iter):
  solucion = crear_solucion(Nodos)

  mejor_solucion = solucion
  mejor_distancia = distancia_total(solucion, problem)

  cont = 0
  while cont < iter:
    solucion_f = busqueda_local(solucion, problem)

    distancia_f = distancia_total(solucion_f, problem)
    if distancia_f < mejor_distancia:
      mejor_solucion = solucion_f
      mejor_distancia = distancia_f

    solucion = crear_solucion(Nodos)
    cont += 1

  print("Distancia:", mejor_distancia)
  return solucion

In [20]:
sol = busqueda_multiarranque(problem, 10)

En la iteracion  31 , la mejor solución encontrada es: [0, 38, 22, 30, 28, 2, 27, 3, 1, 16, 14, 39, 21, 24, 40, 10, 26, 5, 6, 7, 17, 31, 35, 36, 37, 15, 19, 13, 4, 29, 8, 9, 23, 41, 25, 11, 12, 18, 32, 34, 33, 20]
Distancia     : 1940
En la iteracion  35 , la mejor solución encontrada es: [0, 27, 2, 28, 29, 10, 25, 1, 7, 37, 15, 4, 8, 9, 21, 40, 24, 39, 30, 14, 16, 19, 13, 11, 12, 18, 26, 5, 6, 3, 32, 20, 33, 34, 38, 22, 23, 41, 31, 35, 36, 17]
Distancia     : 2031
En la iteracion  30 , la mejor solución encontrada es: [0, 36, 35, 20, 33, 34, 38, 22, 29, 26, 5, 13, 19, 15, 37, 17, 31, 1, 3, 27, 2, 28, 30, 32, 4, 6, 7, 16, 14, 18, 12, 11, 25, 10, 41, 21, 40, 24, 39, 9, 23, 8]
Distancia     : 1786
En la iteracion  37 , la mejor solución encontrada es: [0, 17, 35, 36, 37, 15, 16, 14, 19, 7, 23, 41, 10, 26, 6, 5, 13, 18, 12, 11, 25, 9, 39, 21, 40, 24, 22, 38, 34, 33, 20, 31, 32, 30, 29, 8, 28, 27, 2, 3, 4, 1]
Distancia     : 1742
En la iteracion  30 , la mejor solución encontrada es: [0, 3

In [21]:
print(sol)

[0, 30, 11, 4, 8, 1, 31, 41, 5, 15, 39, 38, 12, 27, 37, 32, 34, 33, 19, 36, 24, 9, 20, 35, 7, 3, 2, 14, 13, 22, 21, 29, 28, 6, 18, 40, 16, 10, 25, 17, 26, 23]


#SIMULATED ANNEALING

In [22]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):
  i, j = sorted(random.sample(range(1, len(solucion)), 2))
  return solucion[:i] + [solucion[j]] + solucion[i+1: j] + [solucion[i]] + solucion[j+1:]
  
def genera_vecina_nueva(solucion, problem):
  distancia_mayor_a = -float('inf')
  pos_a = 0
  for i in range(len(solucion) - 1):
    dist = distancia(solucion[i], solucion[i + 1], problem)
    if dist > distancia_mayor_a:
      pos_a = i
      distancia_mayor_a = dist

  distancia_mayor_b = -float('inf')
  pos_b = 0
  for i in range(len(solucion) - 1):
    dist = distancia(solucion[i], solucion[i + 1], problem)
    if dist > distancia_mayor_b and dist < distancia_mayor_a:
      pos_b = i
      distancia_mayor_b = dist

  pos_a, pos_b = (pos_a, pos_b) if pos_a < pos_b else (pos_b, pos_a)
  return solucion[:pos_a] + [solucion[pos_b]] + solucion[pos_a + 1: pos_b] + [solucion[pos_a]] + solucion[pos_b + 1:]
 
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() < math.exp(-1 * d / T):
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T * 0.99

In [23]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina = genera_vecina_nueva(solucion_referencia, problem)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es", mejor_solucion)
  print("con una distancia total de", mejor_distancia)
  return mejor_solucion

In [24]:
sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 22, 2, 37, 29, 24, 40, 11, 30, 1, 16, 12, 17, 20, 35, 19, 39, 28, 3, 13, 14, 36, 7, 33, 15, 5, 26, 9, 10, 8, 27, 6, 32, 4, 25, 38, 34, 31, 18, 23, 21, 41]
con una distancia total de 4006


¿Se puede mejorar con otra elección no tan aleatoria (función genera_vecina_aleatorio())?

La nueva solución propuesta era encontrar las dos ciudades con distancias más largas en toda la solución, e intercambiarlas entre sí, para tratar de reducir las dos distancias más largas y encontrar distancias más pequeñas. Sin embargo, utilizar esta heurística no mejora significativamente el resultado de una selección aleatoria porque incrementa la complejidad y el tiempo de ejecución, y aún así, no se obtienen mejores resultados, sino que se obtienen distancias más largas.